In [1]:
#必要なディレクトリを作成する

import os

# ディレクトリのリストを定義
dir_paths = ["./data/output", 
             "./img/card_img_1", 
             "./img/card_img_2",
             "./img/fire_img"
            ]

# 各ディレクトリについて確認し、存在しない場合にディレクトリを作成
for dir_path in dir_paths:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


# カード2枚の組み合わせ一覧のcsvファイルを作成
./data/output/combolist.csvに出力

In [2]:
import pandas as pd
import itertools
import os

# パスを定数として定義
INPUT_PATH = './data/input/cardlist.csv'
OUTPUT_PATH = './data/output/combolist.csv'

# 出力先ディレクトリの確認と作成
output_dir = os.path.dirname(OUTPUT_PATH)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# csvファイルからカードリストを読み込む
df_card = pd.read_csv(INPUT_PATH)

# カードの組み合わせのデータフレームを作成
df_combo = pd.DataFrame(list(itertools.product(df_card["日本語名"].tolist(),df_card["日本語名"].tolist())),columns=["発火するカード","発火させるカード"])

# 同じカードの組み合わせの行を削除
df_combo = df_combo[df_combo["発火するカード"]!=df_combo["発火させるカード"]].reset_index(drop=True)

# インデックスの名前を設定
df_combo.index.name = 'No'

# 組み合わせリストをcsvとして出力
df_combo.to_csv(OUTPUT_PATH)

# データフレームを表示
df_combo


,発火するカード,発火させるカード
No,,
0,ヒーローシューター,ボールドマーカー
1,ヒーローシューター,わかばシューター
2,ヒーローシューター,シャープマーカー
3,ヒーローシューター,プロモデラーMG
4,ヒーローシューター,スプラシューター
...,...,...
43467,ドライブワイパーデコ,フィンセント
43468,ドライブワイパーデコ,キャンピングシェルターソレーラ
43469,ドライブワイパーデコ,バレルスピナーデコ


# カード2枚の組み合わせから全発火パターンのcsvファイルを作成
上から順に実行

./data/output/combofire.csvに出力

処理時間は1,2時間程度

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import copy

class TableturfCard:
    def __init__(self, img):
        self.image = img
        self.height, self.width = img.shape
    
    def show(self):
        """
        表示用
        """
        plt.imshow(self.image)
        plt.colorbar()
        plt.show()  

    def get_sp_indices(self, card_type):
        """
        SPマスのインデックスを取得 (引数 1: 発火するカード, 2: 発火させるカード)
        """
        sp_value = 2 if card_type == 1 else 4
        return np.argwhere(self.image == sp_value)
    
    def judge_fire(self, card_type):
        """
        発火しているか確認 (引数 1: 発火するカード, 2: 発火させるカード)
        """
        
        # SPマスのインデックスを取得
        sp_indices = self.get_sp_indices(card_type)
        if len(sp_indices) == 0:
            return False

        i, j = sp_indices[0]
        surrounding_array = self.image[max(i-1,0):i+2, max(j-1,0):j+2]
        return np.all(surrounding_array!=0) and surrounding_array.shape==(3,3)


    def judge_put(self, other, h, w):
        """
        カードが置ける条件を全て満たしているか確認
        """
        put_result = self.put(other, h, w)
        return self.judge_overlap(other, h, w) and put_result.judge_card_bond() and put_result.judge_wall_bond()
    
    
    def judge_overlap(self, other, h, w):
        """
        カードが置けるか確認
        """
        # カードが重ならない場合はTrue, 重なる場合はFalseを返す
        return np.all(self.image[h:other.height+h,w:other.width+w]*other.image==0)
        
    
    def judge_card_bond(self):
        """
        カード同士が結合しているか確認
        """
        
        # 5(=壁)を0に置き換えます。
        img_cp = self.image.copy()
        img_cp[img_cp == 5] = 0
        
        _, img_cp_bin = cv2.threshold(img_cp, 0, 1, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(img_cp_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
        
        # 輪郭が1つになっている場合は結合していると判断しTrueを返す、輪郭が1つ以外の場合はFalseを返す
        return len(contours) == 1
    
    def judge_wall_bond(self):
        """
        カードが壁と結合しているか確認(壁がない場合はTrue)
        """    
        # 3, 4(=発火させるカード)を0に置き換えます。
        img_cp = self.image.copy()
        img_cp[img_cp == 3] = 0
        img_cp[img_cp == 4] = 0
        
        _, img_cp_bin = cv2.threshold(img_cp, 0, 1, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(img_cp_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
        
        # 輪郭が1つになっている場合は結合していると判断しTrueを返す、輪郭が1つ以外の場合はFalseを返す
        return len(contours) == 1


    def judge_touch_wall(self, card_type): 
        """
        SPマス周辺に壁があるか確認 (引数 1: 発火するカード, 2: 発火させるカード)
        """        
        # 発火するカードのSPマス(=2)が存在する位置を探す
        sp_indices = self.get_sp_indices(card_type)
        if len(sp_indices) == 0:
            return False
        
        i, j  =  sp_indices[0]
        # 周りの領域をスライシングする
        region = self.image[max(0, i-1):i+2, max(0, j-1):j+2]
        # SPマス(=2)の周りに壁(=5)が存在するか確認する
        return 5 in region

    def put(self, other, h, w):
        """
        カードを置く　(発火させるカードの各ピクセル値が1と2の場所をそれぞれ3と4に置換します。)
        """
        # カードの値のマッピング
        mapping = {1: 3, 2: 4}

        # マッピング関数のベクトル化
        vectorized_mapping = np.vectorize(mapping.get)

        # other.imageのマッピング
        other_im = vectorized_mapping(other.image, other.image).astype(np.uint8)

        imagecp = self.image.copy()
        imagecp[h:other.height+h, w:other.width+w] += other_im

        return TableturfCard(imagecp)


    def padding(self, top, bottom, left, right):
        """
        余白を作る
        """
        return  TableturfCard(cv2.copyMakeBorder(self.image.copy(), top, bottom, left, right, cv2.BORDER_CONSTANT, value=0))

    def cut(self, pad):
        """
        余白を削る (引数には整数を指定。カット後に残す余白のピクセルサイズを示す。)
        """

        # 壁は無視する
        img_cp = self.image.copy()
        img_cp[img_cp == 5] = 0
        
        _, binary_image = cv2.threshold(img_cp, 0, 1, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
        x1, y1, x2, y2 = [], [], [], []

        for i in range(0, len(contours)):
            rect = cv2.boundingRect(contours[i])
            x1.append(rect[0])
            y1.append(rect[1])
            x2.append(rect[0] + rect[2])
            y2.append(rect[1] + rect[3])

        x_min = min(x1)
        y_min = min(y1)
        x_max = max(x2)
        y_max = max(y2)
        return TableturfCard(self.image[y_min-pad:y_max+pad, x_min-pad:x_max+pad])
        

    def rotation(self, rotation_times):
        """
        回転させる (引数: 0: 0°回転、 1: 90°回転、 2: 180°回転、 3: 270°回転、)
        """
        return TableturfCard(np.rot90(self.image.copy(), rotation_times))
            
    def to_bin(self):
        """
        2値化させる (発火するカード(=1, 2)や発火させるカード(=3, 4)や壁(=5)などを1にする)
        """
        _, binary_image = cv2.threshold(self.image.copy(), 0, 1, cv2.THRESH_BINARY)
        return TableturfCard(binary_image )
        
    def add_wall_left(self):
        """
        左に垂直な壁を作る
        """
        new_image = self.image.copy()
        new_image[:,0:1] = 5
        return TableturfCard(new_image)
    
    def add_wall_leftbottom(self, set_type):
        """
        左下に斜め壁を作る (引数 1: カードと壁が密接に結合, 2: SPマスと壁が角同士で結合)
        """
        max_length = max(self.height, self.width)

        # 壁の画像を作成
        wall_shape = np.ones((self.height, self.width), dtype=np.uint8) * 5
        new_self = copy.deepcopy(self)

        # 左下の三角形を1にする
        wn = -max_length + 1
        while wn < max_length:
            if not new_self.judge_overlap(TableturfCard(np.tril(wall_shape, k=wn)), 0, 0):
                if set_type == 1 and new_self.judge_put(TableturfCard(np.tril(wall_shape, k=wn-1)), 0, 0):
                    return new_self.put(TableturfCard(np.tril(wall_shape, k=wn-1)), 0, 0)
                elif set_type == 2 and new_self.judge_put(TableturfCard(np.tril(wall_shape, k=wn-2)), 0, 0):
                    return new_self.put(TableturfCard(np.tril(wall_shape, k=wn-2)), 0, 0)
            wn += 1
        return new_self


    def add_corner_leftbottom(self, set_type):
        '''
        カードの左下を曲がり角で埋める(引数 1: SPマスに左に結合,  2: SPマスの下に結合. 3: SPマスの左下に結合)    
        SPマスが存在しない場合や、曲がり角を埋める操作が成功しない場合、元のカードのコピーをそのまま返す。
        '''
        max_length = max(self.height, self.width) # 画像の高さと幅を取得
        new_self = copy.deepcopy(self) # オリジナルのインスタンスをコピー

        # 壁の画像を作成
        wall_img = np.zeros((new_self.height, new_self.width), dtype=np.uint8)

        sp_indices = new_self.get_sp_indices(1)
        
        # SPマスが存在しない場合、曲がり角を接着させても発火しないので何もしない
        if len(sp_indices) != 0:
            i, j = sp_indices[0]
            if set_type == 1:  # SPマスに左に結合する場合
                wall_img[i:, :j] = 5
            elif set_type == 2:  # SPマスの下に結合する場合
                wall_img[i+1:, :j+1] = 5
            elif set_type == 3:  # SPマスの左下に結合する場合
                wall_img[i+1:, :j] = 5

            if new_self.judge_put(TableturfCard(wall_img), 0, 0):
                return new_self.put(TableturfCard(wall_img), 0, 0)

        return new_self




In [4]:
import numpy as np

def get_unique_list(input_list):
    """
    TableturfCardオブジェクトのリストを受け取り、
    重複（回転による一致も含む）を除いたリストを返す。

    引数:
    input_list (list): TableturfCardオブジェクトのリスト

    戻り値:
    unique_list (list): 重複を除いたTableturfCardオブジェクトのリスト
    """
    unique_list = []
    for i, card in enumerate(input_list):
        # `card`がリスト内の他のカードと回転により一致するかどうかをチェック
        is_duplicate = any(np.array_equal(np.rot90(card.image, k), input_list[j].image) for j in range(i+1, len(input_list)) for k in range(4))

        # `card`が重複していなければ、ユニークリストに追加
        if not is_duplicate:
            unique_list.append(card)
    
    return unique_list


In [5]:
def run_judgeFire(card1, card2, set_type):
    """
    カードが発火する場合のカードの状態をリストで返します。

    引数:
    card1, card2 : 発火するカード, 発火させるカード
    set_type (str) : 壁のタイプ。"壁なし", "縦横壁", "斜め壁1", "斜め壁2", "曲がり角1", "曲がり角2", "曲がり角3"のいずれか

    戻り値:
    results_list (list) : 発火する場合のカードの状態のリスト
    """
    valid_set_types = ["壁なし", "縦横壁", "斜め壁1", "斜め壁2", "曲がり角1", "曲がり角2", "曲がり角3"]
    if set_type not in valid_set_types:
        raise ValueError(f'Invalid set_type: {set_type}. Valid set_types are {valid_set_types}')

    results_list = []
    
    # 発火するカードを回転させて検証
    for rot1 in range(1 if set_type == "壁なし" else 4): # 壁なしの場合、発火するカードは回転させない(重複になるため)
        card1_rc = card1.rotation(rot1).cut(0)
        # 発火させるカードを回転させる
        for rot2 in range(4):
            card2_rc = card2.rotation(rot2).cut(0)

            if set_type == "壁なし": 
                # パディング処理
                card1_rcpw = card1_rc.padding(card2_rc.height+1, card2_rc.height+1, card2_rc.width+1, card2_rc.width+1)
                pass
            else: # 壁ありの場合、壁を追加する
                if set_type == "縦横壁":
                    card1_rcpw = card1_rc.padding(card2_rc.height+1, card2_rc.height+1, 1, card2_rc.width+1) # パディング処理
                    card1_rcpw = card1_rcpw.add_wall_left()
                elif "斜め壁" in set_type:
                    card1_rcpw = card1_rc.padding(card2_rc.height+1, card2_rc.height+1, card2_rc.width+1, card2_rc.width+1) # パディング処理
                    card1_rcpw = card1_rcpw.add_wall_leftbottom(int(set_type[-1]))
                elif "曲がり角" in set_type:
                    card1_rcpw = card1_rc.padding(card2_rc.height+1, card2_rc.height+1, card2_rc.width+1, card2_rc.width+1) # パディング処理
                    card1_rcpw = card1_rcpw.add_corner_leftbottom(int(set_type[-1]))
                # 追加した壁と発火するカードのSPマスが結合していなければ、壁を絡めた発火にならないためスキップ
                if not card1_rcpw.judge_touch_wall(1):
                    break

            # 発火させるカードを1ピクセルごとに走査し、発火するか検証
            for h1 in range(1, card1_rcpw.height - card2_rc.height):
                for w1 in range(1, card1_rcpw.width - card2_rc.width):
                    # 発火させるカードが置ける場合
                    if card1_rcpw.judge_put(card2_rc, h1, w1):
                        # 発火させるカードを置いて発火するならリストに加える
                        card1_put2 = card1_rcpw.put(card2_rc, h1, w1)
                        if card1_put2.judge_fire(1):
                            results_list.append(card1_put2.cut(1)) # 画像出力用にいい感じに余白をカットする
    return results_list


In [6]:
import pandas as pd
import ast

# カードの組み合わせのCSVを読み込み
df_combo = pd.read_csv('data/output/combolist.csv')

# csvファイルからカードリストを読み込む
df_card = pd.read_csv('./data/input/cardlist.csv')

# 結果格納用のDataFrameを作成
# 先に各列のデータ型を指定してからDataFrameを作成
df_fire = pd.DataFrame(
    columns=[
        "組み合わせ番号", 
        'シーケンス番号', 
        '発火するカード', 
        '発火させるカード', 
        '発火成立真偽', 
        '発火同時成立真偽', 
        '発火タイプ', 
        'マス配列'
    ], 
    dtype=object
)
df_fire = df_fire.astype({
    "組み合わせ番号": int, 
    "シーケンス番号": int, 
    "発火成立真偽": bool, 
    "発火同時成立真偽": bool, 
})

# 組み合わせの各行について調査
for index, row in df_combo.iterrows():
    
    print(f"Processing row: {index}, Fire card: {row['発火するカード']}, Trigger card: {row['発火させるカード']}")

    # 発火するカードと発火させるカードを取得
    fire_card = TableturfCard(np.array(ast.literal_eval(df_card.loc[df_card["日本語名"]==row["発火するカード"], "マス配列"].values[0]),dtype=np.uint8))
    triger_card = TableturfCard(np.array(ast.literal_eval(df_card.loc[df_card["日本語名"]==row["発火させるカード"], "マス配列"].values[0]),dtype=np.uint8))

    num = 0 
    # 各発火タイプについて調査
    for t in ["壁なし", "縦横壁", "斜め壁1", "斜め壁2", "曲がり角1", "曲がり角2", "曲がり角3"]:
        # 発火の結果を格納するリスト
        results = run_judgeFire(fire_card, triger_card, t)

        # 発火の結果の重複を排除
        unique_results = get_unique_list(results)
        
        # 結果格納
        for result in unique_results:
            new_row = pd.DataFrame({
                '組み合わせ番号': [index],
                'シーケンス番号': [num],
                '発火するカード': [row["発火するカード"]], 
                '発火させるカード': [row["発火させるカード"]], 
                '発火成立真偽': [result.judge_fire(1)],  
                '発火同時成立真偽': [result.judge_fire(2)], 
                '発火タイプ': [t], 
                'マス配列': [result.image.tolist()]  # リストに変換して格納
            })
            df_fire = pd.concat([df_fire, new_row], ignore_index=True)
            num += 1


print(f"Processed {index+1} rows.")

Processing row: 0, Fire card: ヒーローシューター, Trigger card: ボールドマーカー
Processing row: 1, Fire card: ヒーローシューター, Trigger card: わかばシューター
Processing row: 2, Fire card: ヒーローシューター, Trigger card: シャープマーカー
Processing row: 3, Fire card: ヒーローシューター, Trigger card: プロモデラーMG
Processing row: 4, Fire card: ヒーローシューター, Trigger card: スプラシューター
Processing row: 5, Fire card: ヒーローシューター, Trigger card: .52ガロン
Processing row: 6, Fire card: ヒーローシューター, Trigger card: N-ZAP85
Processing row: 7, Fire card: ヒーローシューター, Trigger card: プライムシューター
Processing row: 8, Fire card: ヒーローシューター, Trigger card: .96ガロン
Processing row: 9, Fire card: ヒーローシューター, Trigger card: ジェットスイーパー
Processing row: 10, Fire card: ヒーローシューター, Trigger card: ノヴァブラスター
Processing row: 11, Fire card: ヒーローシューター, Trigger card: ホットブラスター
Processing row: 12, Fire card: ヒーローシューター, Trigger card: ロングブラスター
Processing row: 13, Fire card: ヒーローシューター, Trigger card: クラッシュブラスター
Processing row: 14, Fire card: ヒーローシューター, Trigger card: ラピッドブラスター
Processing row: 15, Fire card: ヒーロー

KeyboardInterrupt: 

5584 スクイックリンα N-ZAP89  is running..
5585 スクイックリンα .96ガロンデコ  is running..
5586 スクイックリンα ジェットスイーパーカスタム  is running..
5587 スクイックリンα クラッシュブラスターネオ  is running..
5588 スクイックリンα ラピッドブラスターデコ  is running..
5589 スクイックリンα L3リールガンD  is running..
5590 スクイックリンα スプラローラーコラボ  is running..
5591 スクイックリンα スプラチャージャーコラボ  is running..
5592 スクイックリンα スプラスコープコラボ  is running..
5593 スクイックリンα ヒッセン・ヒュー  is running..
5594 スクイックリンα テイオウイカ  is running..
5595 スクイックリンα デコイチラシ  is running..
5596 スクイックリンα マメブキチ&ツブブキチ  is running..
5597 スクイックリンα アネモ  is running..
5598 スクイックリンα エチゼン  is running..
5599 スクイックリンα アジオ  is running..
5600 スクイックリンα ダウニー  is running..
5601 スクイックリンα 踊るギャングスターウツホ  is running..
5602 スクイックリンα サメ使いのフウカ  is running..
5603 スクイックリンα ふしぎせいぶつマンタロー  is running..
5604 スクイックリンα エゾッコリー  is running..
5605 スクイックリンα スペースシューターコラボ  is running..
5606 スクイックリンα Rブラスターエリートデコ  is running..
5607 スクイックリンα S-BLAST92  is running..
5608 スクイックリンα H3リールガンD  is running..
5609 スクイックリンα ワイドローラーコラボ  is running..
5610 スクイックリンα フィンセント 

5807 スプラチャージャー アジオ  is running..
5808 スプラチャージャー ダウニー  is running..
5809 スプラチャージャー 踊るギャングスターウツホ  is running..
5810 スプラチャージャー サメ使いのフウカ  is running..
5811 スプラチャージャー ふしぎせいぶつマンタロー  is running..
5812 スプラチャージャー エゾッコリー  is running..
5813 スプラチャージャー スペースシューターコラボ  is running..
5814 スプラチャージャー Rブラスターエリートデコ  is running..
5815 スプラチャージャー S-BLAST92  is running..
5816 スプラチャージャー H3リールガンD  is running..
5817 スプラチャージャー ワイドローラーコラボ  is running..
5818 スプラチャージャー フィンセント  is running..
5819 スプラチャージャー キャンピングシェルターソレーラ  is running..
5820 スプラチャージャー バレルスピナーデコ  is running..
5821 スプラチャージャー クアッドホッパーホワイト  is running..
5822 スプラチャージャー デュアルスイーパーカスタム  is running..
5823 スプラチャージャー ドライブワイパーデコ  is running..
5824 スプラスコープ ヒーローシューター  is running..
5825 スプラスコープ ボールドマーカー  is running..
5826 スプラスコープ わかばシューター  is running..
5827 スプラスコープ シャープマーカー  is running..
5828 スプラスコープ プロモデラーMG  is running..
5829 スプラスコープ スプラシューター  is running..
5830 スプラスコープ .52ガロン  is running..
5831 スプラスコープ N-ZAP85  is running..
5832 スプラスコープ プライムシューター  is running..
5833 ス

6040 リッター4K プライムシューター  is running..
6041 リッター4K .96ガロン  is running..
6042 リッター4K ジェットスイーパー  is running..
6043 リッター4K ノヴァブラスター  is running..
6044 リッター4K ホットブラスター  is running..
6045 リッター4K ロングブラスター  is running..
6046 リッター4K クラッシュブラスター  is running..
6047 リッター4K ラピッドブラスター  is running..
6048 リッター4K Rブラスターエリート  is running..
6049 リッター4K L3リールガン  is running..
6050 リッター4K H3リールガン  is running..
6051 リッター4K ボトルガイザー  is running..
6052 リッター4K カーボンローラー  is running..
6053 リッター4K スプラローラー  is running..
6054 リッター4K ダイナモローラー  is running..
6055 リッター4K ヴァリアブルローラー  is running..
6056 リッター4K パブロ  is running..
6057 リッター4K ホクサイ  is running..
6058 リッター4K スクイックリンα  is running..
6059 リッター4K スプラチャージャー  is running..
6060 リッター4K スプラスコープ  is running..
6061 リッター4K 4Kスコープ  is running..
6062 リッター4K 14式竹筒銃・甲  is running..
6063 リッター4K ソイチューバー  is running..
6064 リッター4K バケットスロッシャー  is running..
6065 リッター4K ヒッセン  is running..
6066 リッター4K スクリュースロッシャー  is running..
6067 リッター4K オーバーフロッシャー  is running..
6068 リッター4K エクスプロッシャー  is 

...

42595 キャンピングシェルターソレーラ プロモデラーRG  is running..
42596 キャンピングシェルターソレーラ スプラシューターコラボ  is running..
42597 キャンピングシェルターソレーラ プライムシューターコラボ  is running..
42598 キャンピングシェルターソレーラ スペースシューター  is running..
42599 キャンピングシェルターソレーラ ノヴァブラスターネオ  is running..
42600 キャンピングシェルターソレーラ カーボンローラーデコ  is running..
42601 キャンピングシェルターソレーラ ワイドローラー  is running..
42602 キャンピングシェルターソレーラ パブロ・ヒュー  is running..
42603 キャンピングシェルターソレーラ R-PEN/5H  is running..
42604 キャンピングシェルターソレーラ バケットスロッシャーデコ  is running..
42605 キャンピングシェルターソレーラ スプラスピナーコラボ  is running..
42606 キャンピングシェルターソレーラ スパッタリー・ヒュー  is running..
42607 キャンピングシェルターソレーラ ボールドマーカーネオ  is running..
42608 キャンピングシェルターソレーラ シャープマーカーネオ  is running..
42609 キャンピングシェルターソレーラ N-ZAP89  is running..
42610 キャンピングシェルターソレーラ .96ガロンデコ  is running..
42611 キャンピングシェルターソレーラ ジェットスイーパーカスタム  is running..
42612 キャンピングシェルターソレーラ クラッシュブラスターネオ  is running..
42613 キャンピングシェルターソレーラ ラピッドブラスターデコ  is running..
42614 キャンピングシェルターソレーラ L3リールガンD  is running..
42615 キャンピングシェルターソレーラ スプラローラーコラボ  is running..
42616 キャンピングシェルターソレー

42805 バレルスピナーデコ プライムシューターコラボ  is running..
42806 バレルスピナーデコ スペースシューター  is running..
42807 バレルスピナーデコ ノヴァブラスターネオ  is running..
42808 バレルスピナーデコ カーボンローラーデコ  is running..
42809 バレルスピナーデコ ワイドローラー  is running..
42810 バレルスピナーデコ パブロ・ヒュー  is running..
42811 バレルスピナーデコ R-PEN/5H  is running..
42812 バレルスピナーデコ バケットスロッシャーデコ  is running..
42813 バレルスピナーデコ スプラスピナーコラボ  is running..
42814 バレルスピナーデコ スパッタリー・ヒュー  is running..
42815 バレルスピナーデコ ボールドマーカーネオ  is running..
42816 バレルスピナーデコ シャープマーカーネオ  is running..
42817 バレルスピナーデコ N-ZAP89  is running..
42818 バレルスピナーデコ .96ガロンデコ  is running..
42819 バレルスピナーデコ ジェットスイーパーカスタム  is running..
42820 バレルスピナーデコ クラッシュブラスターネオ  is running..
42821 バレルスピナーデコ ラピッドブラスターデコ  is running..
42822 バレルスピナーデコ L3リールガンD  is running..
42823 バレルスピナーデコ スプラローラーコラボ  is running..
42824 バレルスピナーデコ スプラチャージャーコラボ  is running..
42825 バレルスピナーデコ スプラスコープコラボ  is running..
42826 バレルスピナーデコ ヒッセン・ヒュー  is running..
42827 バレルスピナーデコ テイオウイカ  is running..
42828 バレルスピナーデコ デコイチラシ  is running..
42829 バレルスピナーデコ マメブキチ&ツブブキチ  i

43009 クアッドホッパーホワイト ガチアサリ  is running..
43010 クアッドホッパーホワイト もみじシューター  is running..
43011 クアッドホッパーホワイト プロモデラーRG  is running..
43012 クアッドホッパーホワイト スプラシューターコラボ  is running..
43013 クアッドホッパーホワイト プライムシューターコラボ  is running..
43014 クアッドホッパーホワイト スペースシューター  is running..
43015 クアッドホッパーホワイト ノヴァブラスターネオ  is running..
43016 クアッドホッパーホワイト カーボンローラーデコ  is running..
43017 クアッドホッパーホワイト ワイドローラー  is running..
43018 クアッドホッパーホワイト パブロ・ヒュー  is running..
43019 クアッドホッパーホワイト R-PEN/5H  is running..
43020 クアッドホッパーホワイト バケットスロッシャーデコ  is running..
43021 クアッドホッパーホワイト スプラスピナーコラボ  is running..
43022 クアッドホッパーホワイト スパッタリー・ヒュー  is running..
43023 クアッドホッパーホワイト ボールドマーカーネオ  is running..
43024 クアッドホッパーホワイト シャープマーカーネオ  is running..
43025 クアッドホッパーホワイト N-ZAP89  is running..
43026 クアッドホッパーホワイト .96ガロンデコ  is running..
43027 クアッドホッパーホワイト ジェットスイーパーカスタム  is running..
43028 クアッドホッパーホワイト クラッシュブラスターネオ  is running..
43029 クアッドホッパーホワイト ラピッドブラスターデコ  is running..
43030 クアッドホッパーホワイト L3リールガンD  is running..
43031 クアッドホッパーホワイト スプラローラーコラボ  is running..
43

43206 デュアルスイーパーカスタム ハシラ  is running..
43207 デュアルスイーパーカスタム ダイバー  is running..
43208 デュアルスイーパーカスタム ナベブタ  is running..
43209 デュアルスイーパーカスタム テッキュウ  is running..
43210 デュアルスイーパーカスタム キンシャケ  is running..
43211 デュアルスイーパーカスタム グリル  is running..
43212 デュアルスイーパーカスタム ハコビヤ  is running..
43213 デュアルスイーパーカスタム ドロシャケ  is running..
43214 デュアルスイーパーカスタム デンチナマズ  is running..
43215 デュアルスイーパーカスタム ガチヤグラ  is running..
43216 デュアルスイーパーカスタム ガチホコ  is running..
43217 デュアルスイーパーカスタム ガチアサリ  is running..
43218 デュアルスイーパーカスタム もみじシューター  is running..
43219 デュアルスイーパーカスタム プロモデラーRG  is running..
43220 デュアルスイーパーカスタム スプラシューターコラボ  is running..
43221 デュアルスイーパーカスタム プライムシューターコラボ  is running..
43222 デュアルスイーパーカスタム スペースシューター  is running..
43223 デュアルスイーパーカスタム ノヴァブラスターネオ  is running..
43224 デュアルスイーパーカスタム カーボンローラーデコ  is running..
43225 デュアルスイーパーカスタム ワイドローラー  is running..
43226 デュアルスイーパーカスタム パブロ・ヒュー  is running..
43227 デュアルスイーパーカスタム R-PEN/5H  is running..
43228 デュアルスイーパーカスタム バケットスロッシャーデコ  is running..
43229 デュアルスイーパーカスタム スプラスピナーコラボ  is runni

43412 ドライブワイパーデコ コウモリ  is running..
43413 ドライブワイパーデコ カタパッド  is running..
43414 ドライブワイパーデコ ハシラ  is running..
43415 ドライブワイパーデコ ダイバー  is running..
43416 ドライブワイパーデコ ナベブタ  is running..
43417 ドライブワイパーデコ テッキュウ  is running..
43418 ドライブワイパーデコ キンシャケ  is running..
43419 ドライブワイパーデコ グリル  is running..
43420 ドライブワイパーデコ ハコビヤ  is running..
43421 ドライブワイパーデコ ドロシャケ  is running..
43422 ドライブワイパーデコ デンチナマズ  is running..
43423 ドライブワイパーデコ ガチヤグラ  is running..
43424 ドライブワイパーデコ ガチホコ  is running..
43425 ドライブワイパーデコ ガチアサリ  is running..
43426 ドライブワイパーデコ もみじシューター  is running..
43427 ドライブワイパーデコ プロモデラーRG  is running..
43428 ドライブワイパーデコ スプラシューターコラボ  is running..
43429 ドライブワイパーデコ プライムシューターコラボ  is running..
43430 ドライブワイパーデコ スペースシューター  is running..
43431 ドライブワイパーデコ ノヴァブラスターネオ  is running..
43432 ドライブワイパーデコ カーボンローラーデコ  is running..
43433 ドライブワイパーデコ ワイドローラー  is running..
43434 ドライブワイパーデコ パブロ・ヒュー  is running..
43435 ドライブワイパーデコ R-PEN/5H  is running..
43436 ドライブワイパーデコ バケットスロッシャーデコ  is running..
43437 ドライブワイパーデコ スプラスピナーコラボ  is runn

In [9]:
# 結果をCSVに出力
df_fire = df_fire.reset_index(drop=True).rename_axis('No')
df_fire.to_csv("data/output/combofire.csv")

df_fire

,組み合わせ番号,シーケンス番号,発火するカード,発火させるカード,発火成立真偽,発火同時成立真偽,発火タイプ,マス配列
No,,,,,,,,
0,0,0,ヒーローシューター,ボールドマーカー,True,False,壁なし,"[[0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0],..."
1,0,1,ヒーローシューター,ボールドマーカー,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, ..."
2,1,0,ヒーローシューター,わかばシューター,True,False,壁なし,"[[0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0],..."
3,1,1,ヒーローシューター,わかばシューター,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 0],..."
4,3,0,ヒーローシューター,プロモデラーMG,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, ..."
...,...,...,...,...,...,...,...,...
464,279,1,ボールドマーカー,マルチミサイル,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 0, 0, 0],..."
465,280,0,ボールドマーカー,アメフラシ,True,False,壁なし,"[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 3, 0],..."
466,283,0,ボールドマーカー,キューインキ,True,False,曲がり角1,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 3, 0, ..."


# 発火したカードの状態に、「マス数」「長軸」「短軸」などの追加情報を付与する
発火したカードのパターンが10万件以上あるため、処理時間に数十秒かかる

In [10]:
import pandas as pd
import numpy as np
import cv2
import ast

# 既存の発火データを読み込む
df_fire = pd.read_csv('./data/output/combofire.csv', index_col="No")

# 発火したカードの状態に、「長軸」「短軸」「面積」「密度」「穴があるか」の追加情報を付与する関数
def card_info_append(row):
    #発火後のカードを取得
    img = np.array(eval(row['マス配列']), dtype=np.uint8)

    #コピー
    gray=img.copy()
    
    # 壁(=5)を0扱いにする。
    gray[gray == 5] = 0

    # 2値化して発火後のカードの形状を取得
    _, thresh = cv2.threshold(gray, 0, 1, cv2.THRESH_BINARY)

    # マス数を計算する
    count = np.count_nonzero(thresh)
    
    # カードを輪郭抽出する
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)  # 面積でソート
    
    # 穴の有無を判定する
    has_hole = False
    for i in range(len(contours)):
        if hierarchy[0][0][3] == 0:
            has_hole = True
            break
            
    # 長辺、短辺、面積を計算する
    largest_rect = cv2.boundingRect(contours[0])  # 最大の矩形を取得
    x, y, width, height = largest_rect
    long_side = max(width, height)
    short_side = min(width, height)
    area = width * height

    # 密度を計算する
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [contours[0]], 0, 255, -1)
    non_zero_pixels = cv2.countNonZero(mask)
    density = non_zero_pixels / area
    
    # 計算結果をpd.Seriesで返す
    return pd.Series({"マス": count, "長軸": long_side, "短軸": short_side, "面積": area, "密度": density, "穴": has_hole})

# applyを使用して新しいカラムを一気に追加
new_columns = df_fire.apply(card_info_append, axis=1)
df_fire = pd.concat([df_fire, new_columns], axis=1)

# マス配列をリスト形式に変換
df_fire['マス配列'] = df_fire['マス配列'].apply(lambda x: ast.literal_eval(x))

# csvファイル書き出し
df_fire.to_csv("./data/output/combofire_append.csv", index_label="No")

# 結果を表示
df_fire


,組み合わせ番号,シーケンス番号,発火するカード,発火させるカード,発火成立真偽,発火同時成立真偽,発火タイプ,マス配列,マス,長軸,短軸,面積,密度,穴
No,,,,,,,,,,,,,,
0,0,0,ヒーローシューター,ボールドマーカー,True,False,壁なし,"[[0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0],...",21,6,5,30,0.700000,False
1,0,1,ヒーローシューター,ボールドマーカー,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, ...",21,6,5,30,0.700000,False
2,1,0,ヒーローシューター,わかばシューター,True,False,壁なし,"[[0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0],...",16,5,4,20,0.800000,False
3,1,1,ヒーローシューター,わかばシューター,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 0],...",16,5,5,25,0.640000,False
4,3,0,ヒーローシューター,プロモデラーMG,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, ...",19,6,5,30,0.633333,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,279,1,ボールドマーカー,マルチミサイル,True,False,曲がり角3,"[[0, 0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 0, 0, 0],...",21,7,5,35,0.600000,False
465,280,0,ボールドマーカー,アメフラシ,True,False,壁なし,"[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 3, 0],...",21,6,5,30,0.700000,False
466,283,0,ボールドマーカー,キューインキ,True,False,曲がり角1,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 3, 0, ...",21,8,6,48,0.437500,False


# PowerBIに載せるマスの画像を出力する

In [11]:
import numpy as np
import cv2

# カラーマップの定義
# 各キーはカードの種類を表し、値はRGBカラーコードを表す
color_map = {
    0: [0, 0, 0],       # black
    1: [0, 255, 255],   # yellow
    2: [0, 200, 255],   # orange
    3: [0, 255, 200],     # green
    4: [0, 255, 100],   # light green
    5: [100, 100, 100]  # gray
}

# カード画像の配列、カラーマップ、1マスのピクセルサイズ、保存先パスを引数に指定
def create_color_map_image(arr, color_map, cell_size, save_path):
    """
    arr: 2D array, カード画像の2次元配列
    color_map: dict, 使用するカラーマップを定義する辞書
    cell_size: int, 一つのマスを表示するためのピクセル数
    save_path: str, 出力画像の保存先のパス

    機能:
    - arr の各要素に対応する色を color_map から取り出し、RGB画像を作成
    - 作成した画像は save_path に保存される
    """
    # セルの数
    n_rows, n_cols = arr.shape

    # 画像のサイズ
    img_size_rows = cell_size * n_rows
    img_size_cols = cell_size * n_cols

    # 灰色のベース画像を生成
    img = np.ones((img_size_rows, img_size_cols, 3), np.uint8) * 128

    # カード画像に基づいて各セルを塗りつぶす
    for i in range(n_rows):
        for j in range(n_cols):
            # 色を選択
            color = color_map[arr[i, j]]
            # セルを塗りつぶす
            cv2.rectangle(img, (j*cell_size, i*cell_size), ((j+1)*cell_size, (i+1)*cell_size), color, -1)

    # 各セルを白い線で囲む
    for i in range(n_rows+1):
        cv2.line(img, (0, i*cell_size), (img_size_cols, i*cell_size), (100, 100, 100), 1)
    for j in range(n_cols+1):
        cv2.line(img, (j*cell_size, 0), (j*cell_size, img_size_rows), (100, 100, 100), 1)

    # 保存
    cv2.imwrite(save_path, img)


### カード画像保存

In [12]:
# 発火するカード

# 画像出力する1マスあたりのピクセルサイズ
cell_size = 10

df_card = pd.read_csv('data/input/cardlist.csv')

for index, row in df_card.iterrows():
    save_path = "img/card_img_1/" + str(row['No']) + ".jpg"
    create_color_map_image(np.array(eval(row['マス配列']), dtype=np.uint8), color_map, cell_size, save_path)


In [13]:
# 発火させるカード

# カードの値のマッピング
mapping = {1: 3, 2: 4}
# マッピング関数のベクトル化
vectorized_mapping = np.vectorize(mapping.get)

for index, row in df_card.iterrows():
    save_path = "img/card_img_2/" + str(row['No']) + ".jpg"
    img_array = vectorized_mapping(eval(row['マス配列']), eval(row['マス配列'])).astype(np.uint8)
    create_color_map_image(np.array(img_array, dtype=np.uint8), color_map, cell_size, save_path)


### 発火パターン画像の保存
発火したカードのパターンが10万件以上あるため、処理時間は数分ほど

./img/fire_imgディレクトリに「{組み合わせ番号}_{シーケンス番号}.jpg」の名前で出力。(2GB弱)

In [14]:
# 画像出力する1マスあたりのピクセルサイズ
cell_size = 10

df_fire = pd.read_csv('data/output/combofire_append.csv')

for index, row in df_fire.iterrows():
    save_path = "img/fire_img/" + str(row['組み合わせ番号']) + "_" + str(row['シーケンス番号']) + ".jpg"
    create_color_map_image(np.array(eval(row['マス配列']), dtype=np.uint8), color_map, cell_size, save_path)
